In [8]:
import numpy as np
from utils import parse_csi, get_data_files, generate_labels, split_csi
from nn_utils import CSIModel, train, evaluate, ComplexCSIModel


import numpy as np
import torch
from torch.utils.data import DataLoader, random_split, TensorDataset
import torch.nn as nn
import torch.optim as optim

import os 

In [9]:

file_paths = get_data_files(data_dir = '../data/') 
file_paths 

['../data/room_dooropen.csv',
 '../data/room_doorclosed2.csv',
 '../data/room_doorclosed.csv']

In [10]:
macs = ['A0:A3:B3:AA:76:38']

data, found_macs = [], [] 
for file_path in file_paths: 
    csi, mac = parse_csi(file_path, macs = macs)
    csi = split_csi(csi, 3000) 

    data.append(csi)
    found_macs.append(mac)

TypeError: parse_csi() got an unexpected keyword argument 'macs'